In [1]:
import pandas as pd#############

###paths#####
#############
import sys
import os

pwd = os.getcwd()
modellingpath = pwd.rpartition("modelling")[0] + pwd.rpartition("modelling")[1] 
sys.path.append(modellingpath + '/lib')
#############

import pickle

In [3]:
# Specify name of circuit and variant investigated
circuit_n='circuit14'
variant='fitted7'
n_param_sets = 13700000

fit_df = pickle.load( open(modellingpath + '/3954/paper/out/analytical/lsa_dataframes/all_dataframes/lsa_df_%s_variant%s_%sparametersets_balancedSemiBalanced.pkl'%(circuit_n,variant,n_param_sets), "rb"))
print(fit_df['system_class'].value_counts())

simple stable           4081655
complex unstable           6835
hopf                        612
turing I oscillatory          2
turing I hopf                 1
Name: system_class, dtype: int64


In [17]:
nvd = 2
nfe = 5
nda=2
nce=3

Ve3 = 7.9465648854961835



def gfp1_steadystate(OC14, Vf,Kvd): 
    muv = 0.0225 ; kv =  0.0183 ;
    F1 = 1 + Vf*(1/(1+((muv*Kvd)/(kv*OC14 + 1e-8))**nvd ))
    return F1


def rfp1_steadystate(OC14, Ve, Vf,Kvd,Kfe): 
    muv = 0.0225 ; kv =  0.0183 ;
    E1 = 1 + Ve*(1/(1+((1 + Vf*(1/(1+((muv*Kvd)/(kv*OC14 + 1e-8))**nvd )))/(Kfe+1e-8))**nfe))
    
    return E1


def gfp3_steadystate(OC14,  Vd): 
    muv = 0.0225 ; kv =  0.0183 ;
    D3 = 1 + Vd*(1/(1+((muv*Kvd)/(kv*OC14 + 1e-8))**nvd ))

    return D3

def bfp3_steadystate(D,Vc,Kda):
    C3 = 1 + Vc*(1/(1+((D/(Kda+1e-8))**nda)))
    return C3


# def rfp3_steadystate(D,Vc, Kda,Kce): 
# def rfp3_steadystate(D,Vc,Ve, Kda,Kce): 
def rfp3_steadystate(D,Vc, Kda,Kce): 

    E3 = 1 + Ve3*(1/(1+((bfp3_steadystate(D, Vc, Kda)/(Kce+1e-8))**nce)))
    return E3

def steadystate_combined_subcircuits(OC14,Ve, Vf, Kvd, Kfe, Vc, Vd, Kda, Kce):

  F1 = gfp1_steadystate(OC14, Vf,Kvd)
  E1 = rfp1_steadystate(OC14,Ve,Vf,Kvd,Kfe)
  D3 = gfp3_steadystate(OC14, Vd)
  E3 = rfp3_steadystate(OC14,  Vc, Kda,Kce)
  stacked_fluorescence = [F1,E1, D3, E3]

  return stacked_fluorescence


In [15]:
instabilities = ['turing I', 'turing II', 'turing I hopf', 'turing I oscillatory', 'turing II hopf', 'turing semi-hopf']  
instabilities_fit_df = fit_df.loc[fit_df['system_class'].isin(instabilities)]
#drop multindex
instabilities_fit_df.index  = instabilities_fit_df.index.droplevel(-1)

instabilities_fit_df

,Dr,Va,Vb,Ve,Kub,Keb,Kee,muLVA,muASV,nub,...,Kda,Kce,balance,ss_n,ss_list,ss_class,system_class,maxeig,estimated_wvl,complex_dispersion
4187715,0.146703,21.74389,321.614363,8.538026,44.212016,1.517623,0.001,3.595799,1.0,1.0,...,0.73129,2.316786,Semi balanced,1.0,"[3.7898, 12.0271, 3.1751, 1.9061, 1.6213, 1.6143]",stable spiral,turing I oscillatory,0.037464+0.000000j,3.333333,True
6160552,0.033508,401.606776,337.193632,8.563799,110.873775,1.108192,0.001,3.782621,1.0,1.0,...,2.218651,3.060221,Balanced,3.0,"[47.1065, 23.7493, 3.2471, 6.1607, 1.507, 4.0057]",stable spiral,turing I oscillatory,0.152249+0.000000j,2.857143,True
12837401,0.099357,102.692738,431.767123,4.903906,34.045963,0.78697,0.001,3.693107,1.0,1.0,...,1.213817,3.971957,Semi balanced,1.0,"[18.2216, 16.3447, 5.2316, 2.7041, 1.3558, 1.8...",unstable spiral,turing I hopf,0.174278+0.909396j,NaN,True


In [22]:
pfitDict

{'Ve': 8.538026462365124,
 'Vf': 3.571758153811742,
 'Kvd': 26.388959901206217,
 'Kfe': 1.4165582004454278,
 'Vc': 16.95302523649266,
 'Vd': 5.268430929692057,
 'Kda': 0.7312899137021844,
 'Kce': 2.3167863396033366}

In [21]:
parID = 4187715

pfitDict = {}
paramList = ["Ve", "Vf", "Kvd", "Kfe", "Vc", "Vd", "Kda", "Kce"]
for param in paramList:
    pfitDict[param] = instabilities_fit_df.loc[parID, param]
pfitDict

OC14 = [4.42718872e-01, 1.40000000e+00, 4.42718872e+00, 1.40000000e+01,
 4.42718872e+01 ,1.40000000e+02 ,4.42718872e+02, 1.40000000e+03,
 4.42718872e+03 ,1.40000000e+04]

steadystate_combined_subcircuits(OC14, *pfitDict.values())

TypeError: can't multiply sequence by non-int of type 'float'